In [13]:
import yfinance as yf
import pandas as pd
import numpy as np
import vectorbt as vbt
import empyrical as ep
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

# Define tickers and their names
tickers = {
    "^GSPC": "S&P 500",
    "^DJI": "Dow Jones Industrial Average",
    "^IXIC": "NASDAQ Composite",
    "^NYA": "NYSE COMPOSITE (DJ)",
    "^XAX": "NYSE AMEX COMPOSITE INDEX",
    "^BUK100P": "Cboe UK 100",
    "^RUT": "Russell 2000",
    "^FTSE": "FTSE 100",
    "^GDAXI": "DAX PERFORMANCE-INDEX",
    "^FCHI": "CAC 40",
    "^STOXX50E": "ESTX 50 PR.EUR",
    "^N100": "Euronext 100 Index",
    "^BFX": "BEL 20",
    "IMOEX.ME": "MOEX Russia Index",
    "^N225": "Nikkei 225",
    "^HSI": "HANG SENG INDEX",
    "000001.SS": "SSE Composite Index",
    "399001.SZ": "Shenzhen Index",
    "^STI": "STI Index",
    "^AXJO": "S&P/ASX 200",
    "^AORD": "ALL ORDINARIES",
    "^BSESN": "S&P BSE SENSEX",
    "^JKSE": "IDX COMPOSITE",
    "^KLSE": "FTSE Bursa Malaysia KLCI",
    "^NZ50": "S&P/NZX 50 INDEX GROSS (GROSS)",
    "^KS11": "KOSPI Composite Index",
    "^TWII": "TSEC weighted index",
    "^GSPTSE": "S&P/TSX Composite index",
    "^BVSP": "IBOVESPA",
    "^MXX": "IPC MEXICO",
    # "^MERV": "MERVAL",
    "^TA125.TA": "TA-125",
    "^JN0U.JO": "Top 40 USD Net TRI Index",
    "^SET.BK": "Stock Exchange of Thailand",
    "TDEX.BK": "ThaiDEX SET50"
}

# Define start and end dates for historical data fetching
start_date = "2014-07-13"  # Adjusted to approximately 10 years ago
end_date = "2024-07-13"

# Fetch historical data using yfinance
historical_data = yf.download(list(tickers.keys()), start=start_date, end=end_date)['Adj Close']

# Calculate correlation matrix
correlation_matrix = historical_data.corr()

# Find pairs with correlation > 0.7
high_corr_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if correlation_matrix.iloc[i, j] > 0.7:
            pair = (correlation_matrix.columns[i], correlation_matrix.columns[j])
            high_corr_pairs.append(pair)

# Function to perform backtesting for a pair of stocks
def backtest_pair(pair, data):
    stock1, stock2 = pair

    if debug:
        print(f"Backtesting pair: {stock1}/{stock2}")

    if len(data) < 2:
        raise ValueError(f"Not enough data for {pair}")

    # Calculate the gap between the two stocks
    data["Gap"] = data[stock1] - data[stock2]

    # Calculate the rolling mean and standard deviation of the gap
    window = 30  # Rolling window size
    data["Rolling Mean"] = data["Gap"].rolling(window=window).mean()
    data["Rolling Std"] = data["Gap"].rolling(window=window).std()

    # Calculate the Z-score
    data["Z-score"] = (data["Gap"] - data["Rolling Mean"]) / data["Rolling Std"]

    z_score_buying = -1.0 * 2
    z_score_selling = 1.0 * 2

    buy_signal = data["Z-score"] < z_score_buying
    sell_signal = data["Z-score"] > z_score_selling
    close_signal = data["Z-score"].abs() < 0.01

    num_buy_orders = buy_signal.sum()
    num_sell_orders = sell_signal.sum()

    try:
        portfolio_buy = vbt.Portfolio.from_signals(
            close=data[stock2],
            entries=buy_signal,
            exits=close_signal,
            init_cash=10000,
            freq="1D",
        )

        portfolio_sell = vbt.Portfolio.from_signals(
            close=data[stock2],
            entries=sell_signal,
            exits=close_signal,
            init_cash=10000,
            freq="1D",
        )

        # Calculate average number of orders per year
        avg_orders_per_year_buy = num_buy_orders / (len(data) / 252)  # Assuming 252 trading days per year
        avg_orders_per_year_sell = num_sell_orders / (len(data) / 252)

        metrics_buy = {
            "Sharpe Ratio": ep.sharpe_ratio(portfolio_buy.returns()),
            "Sortino Ratio": ep.sortino_ratio(portfolio_buy.returns()),
            "Annual Standard Deviation": ep.annual_volatility(portfolio_buy.returns()),
            "Max Drawdown": ep.max_drawdown(portfolio_buy.returns()),
            "Annual Return": ep.annual_return(portfolio_buy.returns()),
            "Cumulative Return": ep.cum_returns_final(portfolio_buy.returns()),
            "Number of Orders": num_buy_orders,
            "Avg Orders per Year": avg_orders_per_year_buy,
        }

        metrics_sell = {
            "Sharpe Ratio": ep.sharpe_ratio(portfolio_sell.returns()),
            "Sortino Ratio": ep.sortino_ratio(portfolio_sell.returns()),
            "Annual Standard Deviation": ep.annual_volatility(portfolio_sell.returns()),
            "Max Drawdown": ep.max_drawdown(portfolio_sell.returns()),
            "Annual Return": ep.annual_return(portfolio_sell.returns()),
            "Cumulative Return": ep.cum_returns_final(portfolio_sell.returns()),
            "Number of Orders": num_sell_orders,
            "Avg Orders per Year": avg_orders_per_year_sell,
        }

        return {
            "Stock Pair": f"{stock1}/{stock2}",
            "Buy Annual Return": metrics_buy["Annual Return"],
            "Buy Cumulative Return": metrics_buy["Cumulative Return"],
            "Buy Annual Std": metrics_buy["Annual Standard Deviation"],
            "Buy Max Drawdown": metrics_buy["Max Drawdown"],
            "Buy Sharpe Ratio": metrics_buy["Sharpe Ratio"],
            "Buy Sortino Ratio": metrics_buy["Sortino Ratio"],
            "Buy Number of Orders": metrics_buy["Number of Orders"],
            "Buy Avg Orders per Year": metrics_buy["Avg Orders per Year"],
            "Sell Annual Return": metrics_sell["Annual Return"],
            "Sell Cumulative Return": metrics_sell["Cumulative Return"],
            "Sell Annual Std": metrics_sell["Annual Standard Deviation"],
            "Sell Max Drawdown": metrics_sell["Max Drawdown"],
            "Sell Sharpe Ratio": metrics_sell["Sharpe Ratio"],
            "Sell Sortino Ratio": metrics_sell["Sortino Ratio"],
            "Sell Number of Orders": metrics_sell["Number of Orders"],
            "Sell Avg Orders per Year": metrics_sell["Avg Orders per Year"],
        }

    except Exception as e:
        print(f"Error backtesting pair {pair}: {e}")
        return None

results = []

debug = False

for pair in high_corr_pairs:
    try:
        data = yf.download(pair, start=start_date, end=end_date, progress=False)["Adj Close"]
        if not data.empty:
            result = backtest_pair(pair, data)
            if result:
                results.append(result)
        else:
            if debug:
                print(f"No data found for {pair}")
            continue
    except Exception as e:
        if debug:
            print(f"Error fetching data for {pair}: {e}")
        continue

# Convert results to a DataFrame
results_df = pd.DataFrame(results)

# Display the results DataFrame
print("Backtesting Results:")
print(results_df)

# Calculate average row
avg_row = results_df.select_dtypes(include=np.number).mean()

# Create a DataFrame for the average row
avg_df = pd.DataFrame(avg_row).transpose()

# Rename index to "AVG"
avg_df.index = ["AVG"]

# Concatenate average row with results_df
results_df = pd.concat([results_df, avg_df])

# Save results to Excel
results_df.to_excel("backtesting_results_high_corr_with_avg_orders.xlsx")

# Display the updated results DataFrame with average row
print("\nAverage Results:")
results_df


[*********************100%%**********************]  34 of 34 completed


Backtesting Results:
              Stock Pair  Buy Annual Return  Buy Cumulative Return  \
0    000001.SS/399001.SZ          -0.000738              -0.007095   
1         IMOEX.ME/^AORD           0.043754               0.551073   
2         IMOEX.ME/^AXJO           0.029798               0.351162   
3         IMOEX.ME/^BVSP           0.058380               0.785246   
4          IMOEX.ME/^DJI           0.065877               0.925028   
..                   ...                ...                    ...   
238      ^STOXX50E/^TWII           0.097852               1.607466   
239       ^STOXX50E/^XAX           0.049535               0.640155   
240      ^TA125.TA/^TWII           0.000000               0.000000   
241       ^TA125.TA/^XAX           0.000000               0.000000   
242           ^TWII/^XAX           0.086156               1.342100   

     Buy Annual Std  Buy Max Drawdown  Buy Sharpe Ratio  Buy Sortino Ratio  \
0          0.240039         -0.602860          0.117934     

,Stock Pair,Buy Annual Return,Buy Cumulative Return,Buy Annual Std,Buy Max Drawdown,Buy Sharpe Ratio,Buy Sortino Ratio,Buy Number of Orders,Buy Avg Orders per Year,Sell Annual Return,Sell Cumulative Return,Sell Annual Std,Sell Max Drawdown,Sell Sharpe Ratio,Sell Sortino Ratio,Sell Number of Orders,Sell Avg Orders per Year
0,000001.SS/399001.SZ,-0.000738,-0.007095,0.240039,-0.602860,0.117934,0.158203,129.000000,13.372275,-0.006882,-0.064446,0.230219,-0.524330,0.085834,0.116929,156.000000,16.171123
1,IMOEX.ME/^AORD,0.043754,0.551073,0.126289,-0.370920,0.402731,0.542285,47.000000,4.585366,0.030310,0.358061,0.137634,-0.370920,0.286247,0.383027,74.000000,7.219512
2,IMOEX.ME/^AXJO,0.029798,0.351162,0.137702,-0.365305,0.282514,0.382785,45.000000,4.390244,0.038733,0.476266,0.115954,-0.203487,0.385816,0.532546,72.000000,7.024390
3,IMOEX.ME/^BVSP,0.058380,0.785246,0.235155,-0.468158,0.359804,0.500657,27.000000,2.643357,0.089765,1.406179,0.237950,-0.468158,0.481191,0.673063,12.000000,1.174825
4,IMOEX.ME/^DJI,0.065877,0.925028,0.162084,-0.370862,0.475132,0.657448,20.000000,1.948203,0.090027,1.422837,0.169687,-0.370862,0.593351,0.826016,18.000000,1.753382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,^STOXX50E/^XAX,0.049535,0.640155,0.161839,-0.525914,0.380800,0.512309,14.000000,1.367972,0.052431,0.687070,0.176746,-0.525914,0.378558,0.513384,29.000000,2.833656
240,^TA125.TA/^TWII,0.000000,0.000000,0.000000,0.000000,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,0.000000,0.000000
241,^TA125.TA/^XAX,0.000000,0.000000,0.000000,0.000000,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,0.000000,0.000000
242,^TWII/^XAX,0.086156,1.342100,0.198609,-0.525914,0.516425,0.711721,8.000000,0.776879,0.083848,1.291359,0.133713,-0.192832,0.669141,0.980397,3.000000,0.291329


In [2]:
results_df.to_excel("indice_res.xlsx")